In [ ]:
# 모델링 코드 출처 : https://wikidocs.net/40710

In [285]:
#우분투에서 전처리한 민원데이터 가져오기
import pandas as pd
text_1 = pd.read_csv('../따릉따릉/minwon_khaiii_mecab_okt_processed_detokenized.csv', encoding = "utf-8")

In [286]:
del text_1["Unnamed: 0"]

In [287]:
#결측치 제거를 사전에 다 했는데 다시 'nan'가 나오는 문제 -> 다시 dropna()적용
not_null = text_1['detokenized'].dropna()

In [288]:
words = []
for i in range(len(text_1)):
    y = not_null[i].split()
    words.append(y)

In [289]:
len(words)

26406

In [290]:
#TF-IDF 행렬 만들기
#values.astype('U')를 붙여서 'np.nan is an invalid document, expected byte or unicode string' 에러 방지
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000) #상위 2000개의 단어를 보존 
X = vectorizer.fit_transform(not_null.values.astype('U'))  
X.shape # TF-IDF 행렬의 (26670, 2000)크기 확인

(26406, 2000)

In [277]:
#TF-IDF 행렬 만들기
#values.astype('U')를 붙여서 'np.nan is an invalid document, expected byte or unicode string' 에러 방지
#from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizer = TfidfVectorizer(max_features = 2000) #상위 2000개의 단어를 보존 
#X = vectorizer.fit_transform(text_1['detokenized'].values.astype('U'))  
#X.shape # TF-IDF 행렬의 (26670, 2000)크기 확인

In [291]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [292]:
lda_top=lda_model.fit_transform(X)

In [293]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[ 0.10348034  0.10002953  0.10003607 ...  0.10005734  0.10004163
   0.10031407]
 [ 0.10016661  7.63103359  0.10004568 ...  0.10007297  0.10004527
   0.1000516 ]
 [ 0.10049974  0.10003091  8.01120039 ...  0.10006774  0.10003343
   0.10004922]
 ...
 [ 8.91217686  0.10006031  0.10005289 ...  0.10009456  0.10004379
   0.10008339]
 [ 0.10003132  0.10003783  0.10003902 ...  0.10012844 20.47479345
   0.10002798]
 [ 0.10003109  0.10005772  0.10007329 ... 47.17617495  0.10005297
  10.63331421]]
(10, 2000)


In [294]:
terms = vectorizer.get_feature_names() 

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('검색', 171.04), ('감사', 124.2), ('공개', 123.78), ('건의', 92.26), ('대여소', 87.26)]
Topic 2: [('따릉이', 241.14), ('설치', 230.36), ('불편', 170.27), ('답변', 149.71), ('이용', 132.36)]
Topic 3: [('자전거', 251.12), ('고장', 171.78), ('신고', 158.41), ('대여', 120.32), ('답변', 107.24)]
Topic 4: [('반납', 416.7), ('장애', 217.73), ('통신', 198.44), ('처리', 183.09), ('오류', 149.64)]
Topic 5: [('물방울', 105.04), ('코드', 73.99), ('예전', 73.73), ('거치대', 72.44), ('이전', 59.44)]
Topic 6: [('환불', 148.97), ('화면', 141.1), ('해도', 129.02), ('문제', 121.51), ('업데이트', 115.93)]
Topic 7: [('대여소', 262.02), ('설치', 244.74), ('따릉이', 172.59), ('아파트', 162.21), ('출구', 161.32)]
Topic 8: [('지도', 261.73), ('대여소', 253.92), ('위치', 189.45), ('설치', 113.72), ('표시', 106.21)]
Topic 9: [('사용', 232.05), ('번호', 213.03), ('이용', 204.86), ('결제', 201.75), ('대여', 188.97)]
Topic 10: [('센터', 159.27), ('시간', 142.9), ('카드', 127.8), ('등록', 105.1), ('전화', 99.84)]


In [295]:
from gensim import corpora
dictionary = corpora.Dictionary(words)
corpus = [dictionary.doc2bow(text) for text in words]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(1, 1), (3, 1), (5, 1), (7, 1), (8, 1), (9, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [296]:
import gensim
NUM_TOPICS = 10 #10개의 토픽
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.062*"따릉이" + 0.040*"이용" + 0.031*"자전거" + 0.019*"대여소"')
(1, '0.041*"자전거" + 0.013*"도로" + 0.013*"위치" + 0.010*"따릉이"')
(2, '0.048*"답변" + 0.033*"글" + 0.018*"공개" + 0.015*"비"')
(3, '0.051*"반납" + 0.029*"시간" + 0.026*"자전거" + 0.023*"분"')
(4, '0.040*"대여소" + 0.033*"대여" + 0.027*"자전거" + 0.025*"지도"')
(5, '0.095*"설치" + 0.076*"대여소" + 0.019*"아파트" + 0.018*"따릉이"')
(6, '0.021*"시" + 0.021*"시민" + 0.020*"의견" + 0.018*"분"')
(7, '0.036*"장애" + 0.032*"통신" + 0.013*"뭐" + 0.012*"공지"')
(8, '0.025*"자전거" + 0.024*"면" + 0.022*"안장" + 0.015*"까지"')
(9, '0.048*"앱" + 0.024*"겠" + 0.019*"면" + 0.015*"시"')


In [297]:
print(ldamodel.print_topics())

[(0, '0.062*"따릉이" + 0.040*"이용" + 0.031*"자전거" + 0.019*"대여소" + 0.014*"설치" + 0.011*"불편" + 0.011*"면" + 0.010*"시민" + 0.010*"곳" + 0.009*"거치대"'), (1, '0.041*"자전거" + 0.013*"도로" + 0.013*"위치" + 0.010*"따릉이" + 0.010*"그래서" + 0.008*"gps" + 0.007*"조금" + 0.007*"학교" + 0.007*"라" + 0.006*"사람"'), (2, '0.048*"답변" + 0.033*"글" + 0.018*"공개" + 0.015*"비" + 0.015*"사항" + 0.015*"질문" + 0.014*"건" + 0.012*"시" + 0.010*"개선" + 0.009*"겠"'), (3, '0.051*"반납" + 0.029*"시간" + 0.026*"자전거" + 0.023*"분" + 0.021*"결제" + 0.021*"시" + 0.021*"이용" + 0.017*"대여" + 0.015*"전화" + 0.014*"처리"'), (4, '0.040*"대여소" + 0.033*"대여" + 0.027*"자전거" + 0.025*"지도" + 0.021*"앱" + 0.020*"고장" + 0.019*"면" + 0.018*"번호" + 0.016*"신고" + 0.016*"카드"'), (5, '0.095*"설치" + 0.076*"대여소" + 0.019*"아파트" + 0.018*"따릉이" + 0.018*"부탁" + 0.015*"추가" + 0.015*"근처" + 0.013*"건" + 0.013*"요청" + 0.013*"동"'), (6, '0.021*"시" + 0.021*"시민" + 0.020*"의견" + 0.018*"분" + 0.013*"제발" + 0.012*"쓰" + 0.011*"하고" + 0.009*"게시판" + 0.009*"따릉이" + 0.009*"민원"'), (7, '0.036*"장애" + 0.032*"통신" + 0.013*"뭐" + 0.012

In [298]:
#시각화
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\NY\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
